### Meta-data local store

- We use SQLite for now...

In [1]:
import os

os.chdir("..")

In [2]:
import json
from pathlib import Path

from pelican_data_loader.db import initialize_database, Dataset, Person
from sqlmodel import Session, create_engine


In [3]:
initialize_database(path=Path("data/datasets.db"), wipe=True)

2025-07-02 11:46:31,122 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-02 11:46:31,123 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("persondatasetlink")
2025-07-02 11:46:31,123 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 11:46:31,124 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("persondatasetlink")
2025-07-02 11:46:31,124 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 11:46:31,125 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("dataset")
2025-07-02 11:46:31,126 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 11:46:31,126 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("dataset")
2025-07-02 11:46:31,127 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 11:46:31,127 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("person")
2025-07-02 11:46:31,128 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 11:46:31,129 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("person")
2025-07-02 11:46:31,129 INFO sqlalchemy.engine.E

In [4]:
# We need a function to flatten most useful metadata from the croissant jsonld to the Dataset model.
# This will be used to populate the Dataset table in the SQLite database.

metadata = json.loads(Path("data/bird_migration_metadata.json").read_text())

# Dataset object include the most useful flattened metadata and the raw JSON-LD metadata.
test_dataset = Dataset.from_jsonld(metadata)


In [5]:
# We populate the primary source with a best guess based on the url's extension, see `pelican_data_loader.db.guess_primary_url` for details.
test_dataset.primary_source_url

'https://web.s3.wisc.edu/pelican-data-loader/data/bird_migration_data.csv'

In [6]:
test_dataset.creators

[Person(first_name='Jason', last_name='Lo', email='jason.lo@wisc.edu', id=None),
 Person(first_name='Test', last_name='User', email='test.user@example.com', id=None)]

In [7]:
# These are the most useful fields for end-user query. We can extend this later.
test_dataset.model_dump(exclude={"croissant_jsonld"})

{'name': 'Bird Migration Data',
 'description': 'Bird migration data',
 'version': '0.0.1',
 'published_date': '2025-07-02',
 'primary_source_url': 'https://web.s3.wisc.edu/pelican-data-loader/data/bird_migration_data.csv',
 'primary_source_sha256': '85da618b044d8220b5a8c3c7030ff3f35f791e875736ed43115415750a824fbf',
 'license': 'https://choosealicense.com/licenses/mit/',
 'keywords': 'bird, testing',
 'id': None}

In [8]:
# Push the parsed metadata to the database.
with Session(create_engine("sqlite:///data/datasets.db")) as session:
    session.add(test_dataset)
    session.commit()


In [9]:
# Query by primary creator email.
from sqlmodel import select

with Session(create_engine("sqlite:///data/datasets.db")) as session:
    statement = select(Dataset).where(Dataset.creators.any(Person.email == "jason.lo@wisc.edu"))  # type: ignore
    results = session.exec(statement)
    for dataset in results:
        print(dataset)


Dataset(id=1, name=Bird Migration Data, version=0.0.1, published_date=2025-07-02)


In [10]:
# Query by keyword `testing`

with Session(create_engine("sqlite:///data/datasets.db")) as session:
    statement = select(Dataset).where(Dataset.keywords.contains("testing"))  # type: ignore
    results = session.exec(statement)
    for dataset in results:
        print(dataset)

Dataset(id=1, name=Bird Migration Data, version=0.0.1, published_date=2025-07-02)


In [11]:
# Query by description contains `bird`

with Session(create_engine("sqlite:///data/datasets.db")) as session:
    statement = select(Dataset).where(Dataset.description.contains("bird"))  # type: ignore
    results = session.exec(statement)
    for dataset in results:
        print(dataset)

Dataset(id=1, name=Bird Migration Data, version=0.0.1, published_date=2025-07-02)
